In [15]:
import Pyro4
import networkx as nx 
from time import time
from multiprocessing import Process


@Pyro4.expose
class NetworkxServer(object):
    def __init__(self, neighbors_fpath):
        tic = time()
        print("Loading the graph:", neighbors_fpath)
        self.G = nx.read_edgelist(
            neighbors_fpath,
            nodetype=str,
            delimiter="\t",
            data=(('weight',float),))
        print("Loaded in {:f} sec.".format(time() - tic))
                   
    def get_neighbors(self, node):
        return self.G[node]
    
    def get_node(self, node):
        return self.G.nodes[node]
    
    def get_edge(self, node_i, node_j):
        return self.G[node_i][node_j]    
    
    def get_node_list(self):
        return self.G.nodes.keys()
    

def start_pyro_server(networkx_server):
    
    daemon = Pyro4.Daemon()   
    uri = daemon.register(networkx_server)
    print("URI:", uri)
    daemon.requestLoop()
    
    return uri 

    
class NetworxServerManager(object):
    def __init__(self, neighbors_fpath):
        networkx_server = NetworkxServer(neighbors_fpath)

        uri = self._graph_server = Process(
                name='NetworkX RPC server',
                target=start_pyro_server,
                args=(networkx_server,))
        self._graph_server.daemon = True
        self._graph_server.start()
        print(uri)

    def stop(self):
        self._graph_server.terminate()        
    


    
    
neighbors_fpath = "/home/panchenko/sensegram/model/text8.graph"
s = NetworxServerManager(neighbors_fpath)


Loading the graph: /home/panchenko/sensegram/model/text8.graph
Loaded in 74.115192 sec.
<Process(NetworkX RPC server, started daemon)>
URI: PYRO:obj_deed3aba75ff46389e3f104621e10f2f@localhost:43890


{}